# Ames House Price Data - Gradient Boosted Trees (XGBoost)

> Juptyer notebook, running a Julia 0.5.2 kernel, with the help of Machine Learning modules written by the author

*We build an XGBoost model using ?

Estimated 95% confidence interval for log-RMS error of Sale Price predictions for the tuned model: 



## Reading in and transforming  the data
Our gradient boosting algorithm is just a wrapping of the popular XGBoost algorithm (sitting atop [XGBoost.jl](https://github.com/dmlc/xgboost) of the Distributed Machine Learning Community). As this algorithm does not handle categorical features, we one-hot encode after loading and standardizing the data. Standardization is recommended because the effect of the regularization parameters is scale-dependent. 

In [ ]:
push!(LOAD_PATH, pwd()) # Allow loading of modules from current directory 
addprocs(3) # for parallel processing
using Preprocess
import DataFrames: head, readtable, writetable
using Regressors, Validation
import TreeCollections: DataTable

df = readtable("2.cleaned/train_randomized.csv")

const y = collect(df[:target])

x = df[2:end-1]
s = StandardizationScheme(x)
xx = transform(s, x)
t = HotEncodingScheme(xx)
const X = Array(transform(t, xx))

## Parameter tuning and cross-validation

XGBoost has quite a few parameters to tune, and we carried out tuning (not published in this notebook) according to [this post](https://www.analyticsvidhya.com/wp-content/uploads/2016/02/5.-gamma.png) of Aarshay Jain. 


In [ ]:
rgs=XGBoostRegressor(alpha=0.006, lambda=0.0, subsample=0.66,
                     colsample_bytree=0.55, eta=0.01, n=2000, min_child_weight=6.0, max_depth=3)


# determine cross-validation error:
err=cv_error(rgs, X, y, n_folds=12, parallel=true, verbose=false) # 0.11413747553914363

In [1]:
# fit on all train data:
fit!(rgs, X, y)